# Bayes Final Project

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pymc3 as pm
from pymc3 import HalfCauchy, Model, Normal, sample, MvNormal 
import xarray as xr
import graphviz
import scipy
import seaborn as sns
from scipy import stats
from scipy.stats import norm
import arviz as az
az.style.use('arviz-darkgrid')
import theano.tensor as tt
import theano

ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.8 from "/Users/leahhogenmiller/.pyenv/versions/3.8.15/bin/python"
  * The NumPy version is: "1.20.3"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: dlopen(/Users/leahhogenmiller/.pyenv/versions/3.8.15/lib/python3.8/site-packages/numpy/core/_multiarray_umath.cpython-38-darwin.so, 0x0002): tried: '/Users/leahhogenmiller/.pyenv/versions/3.8.15/lib/python3.8/site-packages/numpy/core/_multiarray_umath.cpython-38-darwin.so' (mach-o file, but is an incompatible arc

In [11]:
data = pd.read_csv('diabetes')
data = data.drop(['Unnamed: 0'], axis=1)
data = data.astype(int)

In [12]:
data.head()

,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0,1,1,1,4,1,0,0,0,0,...,1,0,5,1,1,1,0,9,4,3
1,0,0,0,0,2,1,0,0,1,0,...,0,1,3,0,0,0,0,7,6,1
2,0,1,1,1,2,0,0,0,0,1,...,1,1,5,1,1,1,0,9,4,8
3,0,1,0,1,2,0,0,0,1,1,...,1,0,2,0,0,0,0,11,3,6
4,0,1,1,1,1,0,0,0,1,1,...,1,0,2,1,0,0,0,11,5,4


In [13]:
data['Diabetes_012'].value_counts()

0    213703
2     35346
1      4631
Name: Diabetes_012, dtype: int64

In [14]:
data_test = data.iloc[range(1000),:]

In [20]:
with pm.Model() as model:
#     X = theano.shared(data_test['Diabetes_012'].to_numpy())
#     Y = theano.shared(data_test.iloc[:,range(1,22)].to_numpy())
    alpha = pm.Normal('alpha', mu=0, sigma=10, shape=(3,))
    beta = pm.Normal('beta', mu=0, sigma=10, shape=(21,3))
    mu = alpha + pm.math.dot(X, beta)
    p = pm.Deterministic('p', tt.nnet.softmax(mu, -1))
    Y1 = pm.Categorical('obs', p=p, observed=Y)

ImportError: dlopen(/Users/leahhogenmiller/.theano/compiledir_macOS-13.0.1-arm64-i386-64bit-i386-3.8.15-64/tmpbjfkgqpr/m31975b6dadb59d49af3f2b36405378825884299512ee0ed9081be33fa591d30d.so, 0x0002): tried: '/Users/leahhogenmiller/.theano/compiledir_macOS-13.0.1-arm64-i386-64bit-i386-3.8.15-64/tmpbjfkgqpr/m31975b6dadb59d49af3f2b36405378825884299512ee0ed9081be33fa591d30d.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64')), '/System/Volumes/Preboot/Cryptexes/OS/Users/leahhogenmiller/.theano/compiledir_macOS-13.0.1-arm64-i386-64bit-i386-3.8.15-64/tmpbjfkgqpr/m31975b6dadb59d49af3f2b36405378825884299512ee0ed9081be33fa591d30d.so' (no such file), '/Users/leahhogenmiller/.theano/compiledir_macOS-13.0.1-arm64-i386-64bit-i386-3.8.15-64/tmpbjfkgqpr/m31975b6dadb59d49af3f2b36405378825884299512ee0ed9081be33fa591d30d.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64'))

In [ ]:
Xt = shared(data_test['Diabetes_012'].to_numpy())
yt = shared(data_test.iloc[:,range(1,22)].to_numpy())

with pm.Model() as model2:

    β = pm.Normal("β", 0, sigma=1e2, shape=(21, 3))
    a = pm.Flat("a", shape=(3,))
    p = at.nnet.softmax(np.dot(Xt, β) + a, -1)

    observed = pm.Categorical("obs", p=p, observed=yt)

In [ ]:
pm.model_to_graphviz(model)

In [ ]:
pm.model_to_graphviz(model2)

In [ ]:
with model:
    model_trace = pm.sample(1000, cores=4)